### Exercises

#### Question 1

Write some code that generates a file containing containing rows containing the following data:

```
i, fibonacci_i, factorial_i, gcd_fibonacci_i_factorial_i
```

where:
- `i`: integer values from `0` to `100`
- `fibonacci_i`: the `i`th Fibonacci number
- `factorial_i`: the factorial of `i` (`i!`)
- `gcd_fib_i_fact_i`: the greatest common denominator of the `i`th Fibonacci number and `i!` 

Hint: look at the `math.factorial` and `math.gcd` functions in the Python docs

Also make sure to include a header row in your file.

For example, the first few rows in your file should contain this data:

```
i,fib,fact,gcd
0,1,1,1
1,1,1,1
2,2,2,2
3,3,6,3
4,5,24,1
5,8,120,8
```

#### Question 2

Using the file you just generated, write three functions:
- `fib`
- `fact`
- `gcd_fib_fact`

that perform the same calculations as our original `fib` function, the `math` module's `factorial` and the `gcd` of the corresponding fibonacci and factorial numbers, but uses the data that was saved in the file as a cache/lookup mechanism - i.e. just use the numbers in the file if they are available, otherwise make the calculation.

#### Solution to Question 1

The cell below generates a CSV file `fib_fact_gcd.csv` with the required data for `i` from 0 to 100.

In [1]:
from __future__ import annotations

from pathlib import Path
import csv
import math
from typing import Iterable, Tuple

DATA_PATH = Path("fib_fact_gcd.csv")


def fibonacci(n: int) -> int:
    """Return the nth Fibonacci number with F0 = 1, F1 = 1.

    Uses an iterative O(n) algorithm.
    """
    if n < 0:
        raise ValueError("n must be non-negative")
    if n in (0, 1):
        return 1
    prev, curr = 1, 1
    for _ in range(2, n + 1):
        prev, curr = curr, prev + curr
    return curr


def generate_rows(start: int = 0, stop: int = 100) -> Iterable[Tuple[int, int, int, int]]:
    """Generate rows (i, fib(i), i!, gcd(fib(i), i!)) for i in [start, stop]."""
    if start < 0 or stop < start:
        raise ValueError("Require 0 <= start <= stop")
    for i in range(start, stop + 1):
        fib_i = fibonacci(i)
        fact_i = math.factorial(i)
        gcd_i = math.gcd(fib_i, fact_i)
        yield i, fib_i, fact_i, gcd_i


def write_data_file(path: Path = DATA_PATH, *, overwrite: bool = True) -> None:
    """Write the CSV file with header and data rows for i in [0, 100]."""
    if path.exists() and not overwrite:
        raise FileExistsError(f"{path} already exists and overwrite=False")
    with path.open("w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["i", "fib", "fact", "gcd"])
        writer.writerows(generate_rows())


if __name__ == "__main__":
    write_data_file()


#### Solution to Question 2

The cell below loads the CSV file as a cache and defines the `fib`, `fact` and `gcd_fib_fact` functions which use the cached values when available (for `0 \u2264 n \u2264 100`) and fall back to computation otherwise.

In [2]:
import csv
from pathlib import Path
import math
from typing import Dict

DATA_PATH = Path("fib_fact_gcd.csv")


class FibFactGcdCache:
    """In-memory cache backed by the CSV file written in Question 1."""

    def __init__(self, path: Path = DATA_PATH) -> None:
        self.path = path
        self._loaded = False
        self._fib: Dict[int, int] = {}
        self._fact: Dict[int, int] = {}
        self._gcd: Dict[int, int] = {}

    def _load(self) -> None:
        if self._loaded:
            return
        if not self.path.exists():
            raise FileNotFoundError(
                f"Data file {self.path} not found. Run write_data_file() from Question 1 first."
            )
        with self.path.open(newline="", encoding="utf-8") as f:
            reader = csv.DictReader(f)
            for row in reader:
                i = int(row["i"])
                self._fib[i] = int(row["fib"])
                self._fact[i] = int(row["fact"])
                self._gcd[i] = int(row["gcd"])
        self._loaded = True

    def fib(self, n: int) -> int:
        self._load()
        if n in self._fib:
            return self._fib[n]
        # Fallback to the implementation from Question 1
        return fibonacci(n)

    def fact(self, n: int) -> int:
        self._load()
        if n in self._fact:
            return self._fact[n]
        return math.factorial(n)

    def gcd_fib_fact(self, n: int) -> int:
        self._load()
        if n in self._gcd:
            return self._gcd[n]
        return math.gcd(self.fib(n), self.fact(n))


_cache = FibFactGcdCache()


def fib(n: int) -> int:
    """Return Fibonacci(n) using CSV cache for 0 <= n <= 100 when available."""
    if n < 0:
        raise ValueError("n must be non-negative")
    return _cache.fib(n)


def fact(n: int) -> int:
    """Return n! using CSV cache for 0 <= n <= 100 when available."""
    if n < 0:
        raise ValueError("n must be non-negative")
    return _cache.fact(n)


def gcd_fib_fact(n: int) -> int:
    """Return gcd(Fibonacci(n), n!) using CSV cache for 0 <= n <= 100 when available."""
    if n < 0:
        raise ValueError("n must be non-negative")
    return _cache.gcd_fib_fact(n)


if __name__ == "__main__":
    # Small demo / sanity check
    for i in range(6):
        print(i, fib(i), fact(i), gcd_fib_fact(i))


0 1 1 1
1 1 1 1
2 2 2 2
3 3 6 3
4 5 24 1
5 8 120 8
